In [ ]:
from keras.models import model_from_json  #gerekli kütüphaneleri (paketleri) tanımlıyoruz
import numpy as np
import time
import keyboard 
from mss import mss
from PIL import Image
"""  https://trex-runner.com/  """
mon = {'top':535, 'left':701, 'width':250, 'height':115}   #ilgili alanın koordinatlarını veriyoruz
sct = mss()

width = 125 #width ,height değerlerini giriyoruz
height = 50

#modeli yüklüyoruz
model = model_from_json(open('model.json','r').read())
model.load_weights('trex_weight.h5')

#down = 0 ,right = 1, up = 2
labels  = ['Down','Right','Up']

framerate_time = time.time() 
counter = 0 #sayaç oluşturuyoruz
i = 0  #frame sayısı olması için tanımlıyoruz
delay = 0.25 #bir komut verildikten 0.4 saniye sonra verilebilir ikinci komut
key_down_pressed = False

while True:  #sonsuz bir döngü oluşturuyoruz
    img = sct.grab(mon) 
    im = Image.frombytes('RGB',img.size,img.rgb)
    im2 = np.array(im.convert('L').resize((width,height))) #yeniden boyutlandırıp im2 değişkenine atıyoruz
    im2 = im2 /255 #normalize ediyorum yani 0-1 aralığına sokuyorum 

    X = np.array([im2]) #im2 değişkenindeki resmi array'e çeviriyoruz
    X = X.reshape(X.shape[0],width,height,1) 

    r =  model.predict(X) #modeli kullanarak prediction(tahmin) işlemi yap ve bunu r değişkenine eşitle
    result = np.argmax(r) # r deki çıktılardan en büyüğünün index numarasını döndürdük [0.1,0.5,0.4]

    if result == 0: #eğer ki tahmin 0 sa yani down işlemi ise 
        keyboard.press(keyboard.KEY_DOWN) #down tuşuna bas
        key_down_pressed= True #ve tuş basıldı
    
    elif result == 2: #eğer ki tahmin 2 ise yani up

        if key_down_pressed:  #eğer tuş hala basılıysa (basılı kaldıysa)
            keyboard.release(keyboard.KEY_DOWN)

        time.sleep(delay)#delay suresi kadar sistemi uyut
        keyboard.press(keyboard.KEY_UP) # up tuşuna bas 

        if i < 1500: #oyun 1500. frame kadar normal akıyor 1500'den sonra oyun hızlanıyor 
            time.sleep(0.3)  #havadayken diğer tuşlara basmaması için sistemi 0.3 saniye uyuttuk 
        elif i > 1500 and i < 5000: #oyun hızlandığı için 0.2 saniye olarak ayarlıyoruz
            time.sleep(0.2)
        else:
            time.sleep(0.17)

        keyboard.press(keyboard.KEY_DOWN) #DOWN tuşuna bastıktan sonra basılı kalıyor bu yüzden 
        keyboard.release(keyboard.KEY_DOWN)   #release ile tuşu basılı olma konumundan kurtarıyoruz
    counter +=1  #sayacı bir artırıyourz
    if (time.time() - framerate_time) > 1: #zaman farkı 1 den buyukse 

        counter = 0 #sayacı ve zamanı sıfırlıyoruz
        framerate_time = time.time()

        if i <= 1500: #frame 1500 den küçük oldugu her an delayı 0.003 azalt
            delay -= 0.003
        else: #frame 1500 den büyük oldugu her an delayı 0.005 azalt
            delay -= 0.005

        if delay < 0: #delay < 0 dan kucukse sıfırla 
            delay = 0 

        print('------------------------')
        print('Down: {} \nRight: {} \nUp: {} \n'.format(r[0][0],r[0][1],r[0][2])) #tahmin değerlerini yazdır
        i += 1      #ve frame değerini bir artır